In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import glob
import sys
sys.path.append('/Users/ChaseWeaver/Desktop/Projects/iCUB-master/') 
import iCUB

# Cleaning Host Data

In [3]:
def clean_host_tsv(df):
    """
    
    """
    
    initial_shape = df.shape
    df = df.reset_index(drop = True)
    
    #Ensure each locus tag is only used once
    df['locus_tag'] = df['qualifiers'].str.split('locus_tag=', n=1, expand=True)[1]\
                            .str.split(';', n=1, expand=True)[0]
    df = df.drop_duplicates(subset = ["locus_tag"], keep = False)
    
    #ensure that only genes with a total length divisible by 3 are used
    df = df[(df['stop']-df['start']) %3 == 2]
    
    #filter out prophages from dataframe
    filter_word = 'phage'
    virus_genes = df[(df['qualifiers'].str.contains(filter_word)==True)]
    df = df[(df['qualifiers'].str.contains(filter_word)==False)]
    
    #adjust for codon bias
    for index in df.index:
        nt_seq = df.at[index, "coding_sequence"]
        if len(nt_seq)%3 == 0:
            df.at[index, "iCUB"] = iCUB.iCUB_Calculator(nt_seq).get_iCUB()
    
    df = df[df['iCUB'].isnull()==False]
    df =df[df['energy_binding'].isnull()==False]
    
    return df

# Clean Virus data

In [ ]:
def clean_virus_tsv(df):
    """
    
    """  
    initial_shape = df.shape
    df = df.reset_index(drop = True)
    
    #Ensure each locus tag is only used once
    df['viral_id'] = df['qualifiers'].str.split('ID=', n=1, expand=True)[1]\
                            .str.split(';', n=1, expand=True)[0]
    
    df = df.drop_duplicates(subset = ["viral_id"], keep = False)
    
    #ensure that only genes with a total length divisible by 3 are used
    df = df[(df['stop']-df['start']) %3 == 2]
    
    #adjust for codon bias
    for index in df.index:
        nt_seq = df.at[index, "coding_sequence"]
        if len(nt_seq) != nt_seq.count('A') + nt_seq.count('T') + nt_seq.count('C') + nt_seq.count('G'):
            continue
        
        if len(nt_seq)%3 != 0:
            continue
            
        df.at[index, "iCUB"] = iCUB.iCUB_Calculator(nt_seq).get_iCUB()           

    df = df[df['iCUB'].isnull()==False]
    df = df[df['energy_binding'].isnull()==False]
    return df